In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nba_api.stats import endpoints
from nba_api.stats.endpoints import playbyplayv2, boxscoresummaryv2, playergamelogs, shotchartdetail
from nba_api.stats.static import players, teams
import time

In [2]:
# eh
# gets the shots of player's teammates in a game's quarter
# @param int player_id id of player in nba_api 
# @param bool home_team if the player's team is the home team in a game
# @param list teammates_ids list of teammates of interests' ids
# @param dataframe pbp_data play-by-play data of a quarter
# @param bool starting if the player is starting
# @return dict teammate id to 2pm, 2pa, 3pm, 3pa during player's time on court
def get_shots_in_quarter(player_id, home_team, teammates_ids, pbp_data, starting):
    opd = {}
    for e in teammates_ids:
        opd[e] = [0,0,0,0]
    
    p_in = starting
    which_desc = "HOMEDESCRIPTION" if home_team else "VISITORDESCRIPTION"
    for _, row in pbp_data.iterrows():
        desc = row[which_desc]
        clock = row["PCTIMESTRING"]
        event_type = row["EVENTMSGTYPE"]
        p1 = row["PLAYER1_ID"]
        if (desc!= None):
            if event_type == 8:
                sub_out = row["PLAYER1_ID"]
                sub_in = row["PLAYER2_ID"]
                if sub_in == player_id:
                    p_in = True
                if sub_out == player_id:
                    p_in = False
            if (p_in):
                #made shot
                if ((event_type == 1) and (p1 in teammates_ids)):
                    if " 3PT " in desc:
                        opd[p1][3] = opd[p1][3] + 1 
                        opd[p1][2] = opd[p1][2] + 1 
                    else:
                        opd[p1][0] = opd[p1][0] + 1 
                        opd[p1][1] = opd[p1][1] + 1 
                #missed shot
                if ((event_type == 2) and (p1 in teammates_ids)):
                    if " 3PT " in desc:
                        opd[p1][3] = opd[p1][3] + 1 
                    else:
                        opd[p1][1] = opd[p1][1] + 1 
                
    return(opd)

In [3]:
# eh
# checks if player started a quarter
# @param int player_id id of player in nba_api 
# @param dataframe pbp_data play-by-play data of a quarter
# @param bool home_team if the player's team is the home team in a game
# @return bool if the player started that quarter
def check_start(player_id, pbp_data, home_team):
    which_desc = "HOMEDESCRIPTION" if home_team else "VISITORDESCRIPTION"
    filtered_data = pbp_data[pbp_data[which_desc].notna()]
    for _,row in filtered_data.iterrows():
        if row["EVENTMSGTYPE"] == 8:
            if player_id == row["PLAYER1_ID"]:
                return True
            elif player_id == row["PLAYER2_ID"]:
                return False
        if (row["PLAYER1_ID"] == player_id) or (row["PLAYER2_ID"] == player_id) or (row["PLAYER3_ID"] == player_id):
            return True
    return False

In [4]:
# eh
# gets all shots of interest from a game
# @param int player_id id of player in nba_api 
# @param bool home_team if the player's team is the home team in a game
# @param list teammates_ids list of teammates of interests' ids
# @param int game_id the id of the game of interest
# @return dict all shots of interest in the game
def get_team_shots_game(player_id, home_team, teammates_ids, game_id):
    master_shots = {}
    for e in teammates_ids:
        master_shots[e] = [0,0,0,0]
    for period in range(1,6):
        pbp_data = playbyplayv2.PlayByPlayV2(game_id=game_id, 
                                        start_period = period, 
                                        end_period = period).get_data_frames()[0]
        period_dict = get_shots_in_quarter(player_id, home_team, teammates_ids , pbp_data, check_start(player_id, pbp_data, home_team))
        for e in teammates_ids:
            master_shots[e] = [sum(x) for x in zip(master_shots[e], period_dict[e])]
    return master_shots

In [5]:
# eh
# gets game_ids of all games the player plays in during a season
# @param int player_id id of player in nba_api 
# @param String season the season of interest
# @return list with game_ids
def get_player_game_ids(player_id, season):
    print("gotten ids")
    game_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable=season)
    games_data = game_logs.get_data_frames()[0]
    game_ids = games_data['GAME_ID'].tolist()
    return game_ids

In [6]:
# eh
# gets all shots of interest from a season
# @param int player_id id of player in nba_api 
# @param int team_id id of the team
# @param list teammates_ids list of teammates of interests' ids
# @param String season the season of interest
# @return dict all shots of interest in the season
def get_team_shots_season(player_id, team_id, teammates_ids, season):
    season_shots_d = {}
    for e in teammates_ids:
        season_shots_d[e] = [0,0,0,0]
    game_num = 0
    print("started")
    for game_id in get_player_game_ids(player_id, season):
        game_num += 1
        print("started game " + str(game_num))
        home_team = (team_id == boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id).game_summary.get_dict()['data'][0][6])
        gd = get_team_shots_game(player_id, home_team, teammates_ids, game_id)
        for e in teammates_ids:
            season_shots_d[e] = [sum(x) for x in zip(season_shots_d[e], gd[e])]
        print("processed game " + str(game_num))
        time.sleep(5)
    return season_shots_d

In [7]:
def get_top_x_teammates_shooting_splits(player_id, season, number_of_teammates): 

    # find team + roster
    team_abbreviation = find_player_team(player_id, season)
    team = teams.find_team_by_abbreviation(team_abbreviation)
    team_id = team['id']
    team_roster_data = endpoints.commonteamroster.CommonTeamRoster(team_id=team_id, season = season).get_data_frames()[0]
    time.sleep(1.5)

    # data analysis for top x teammates
    player_minutes = []
    for _, player in team_roster_data.iterrows():
        if player['PLAYER_ID'] != player_id:
            player_id = player['PLAYER_ID']
            player_name = player['PLAYER']
            game_log = endpoints.playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
            total_minutes = game_log['MIN'].sum()
            player_minutes.append({'name': player_name, 'minutes': total_minutes, 'id': player_id})
            time.sleep(1.5)
    
    minutes_df = pd.DataFrame(player_minutes)
    top_x = minutes_df.sort_values('minutes', ascending=False).head(number_of_teammates)
    
    top_x_id = []

    for _, player in top_x.iterrows():
        top_x_id.append((player['id']))

    # get shooting splits
    top_x_shooting_splits = {}
    
    for top_x_player_id in top_x_id:
        top_x_player_stats = endpoints.playerdashboardbygeneralsplits.PlayerDashboardByGeneralSplits(player_id=top_x_player_id, season=season).get_data_frames()[0]
        three_pt_percent = top_x_player_stats['FG3_PCT'][0]
        two_pt_percent = round((top_x_player_stats['FGM'][0] - top_x_player_stats['FG3M'][0]) / (top_x_player_stats['FGA'][0] - top_x_player_stats['FG3A'][0]), 3)
        two_three_array = [two_pt_percent, three_pt_percent]
        top_x_shooting_splits[top_x_player_id] = two_three_array
        time.sleep(1.5)
    
    return top_x_shooting_splits



def find_player_team(id, season): # helper func 1
    player_info = endpoints.playercareerstats.PlayerCareerStats(player_id=id, per_mode36='Totals').get_data_frames()[0]
    time.sleep(1.5)  
    
    season_info = player_info[player_info['SEASON_ID'] == season]
    
    if not season_info.empty:
        team_abbreviation = season_info['TEAM_ABBREVIATION'].iloc[0]
        return team_abbreviation
    else:
        return None

In [8]:
def everything(player_name, season, nt = 8):
    player_id = players.find_players_by_full_name(player_name.lower())[0]['id']
    dsplits = get_top_x_teammates_shooting_splits(player_id, season, nt)
    teammates_ids = list(dsplits.keys())
    print(list(dsplits.values()))
    team_id = find_player_team(player_id, season)
    shots_dict_season = get_team_shots_season(player_id, team_id, teammates_ids, season)
    print(list(shots_dict_season.values()))

In [9]:
name = "trae young"
season = "2020-21"
everything(name, season)

[[0.51, 0.363], [0.491, 0.343], [0.594, 0.0], [0.615, 0.399], [0.453, 0.321], [0.414, 0.399], [0.522, 0.438], [0.466, 0.406]]
started
gotten ids
started game 1
processed game 1
started game 2
processed game 2
started game 3
processed game 3
started game 4
processed game 4
started game 5
processed game 5
started game 6
processed game 6
started game 7
processed game 7
started game 8
processed game 8
started game 9
processed game 9
started game 10
processed game 10
started game 11
processed game 11
started game 12
processed game 12
started game 13
processed game 13
started game 14
processed game 14
started game 15
processed game 15
started game 16
processed game 16
started game 17
processed game 17
started game 18
processed game 18
started game 19
processed game 19
started game 20
processed game 20
started game 21
processed game 21
started game 22
processed game 22
started game 23
processed game 23
started game 24
processed game 24
started game 25
processed game 25
started game 26
process